In [5]:
def readfasta(filename):
    '''Reading in several fasta files
       yield each pair of header and dna seperate
    '''
    # Try to open the file, error if file does not exsist
    try: file = open(filename, "r")
    except FileNotFoundError as errormessage:
        sys.exit(f"The file '{filename}' could not be found, error: {errormessage}")

    # Extract the dna, and the headers.
    oldheader = "FIRST HEADER"
    for line in file:
        line = line.strip()
        #if line is header yield dna and header except FIRST HEADER
        if line.startswith(">"):    
            newheader = line
            if oldheader != "FIRST HEADER":
                yield dna, oldheader
            dna = ""
            oldheader = newheader
        else:
            dna += line
    # Yield the last header and dna
    yield dna, oldheader
    file.close()
    
import pandas as pd                                                              
import sys
# Should only contain possible plasmids
clusterfile = "/home/bxc755/rasmussen/scratch/ptracker/plasmid_graph/data/vae_clusters_within_radius_with_looners_complete_unsplit_candidate_plasmids.tsv" 
# clusterfile = sys.argv[0]
df_plasmid = pd.read_table(clusterfile, sep="\t")                                             
# len(df.contigname)
df_plasmid = df_plasmid.assign(contigname = ">" + df_plasmid.contigname)

# All contigs
fnafile = "/home/bxc755/rasmussen/scratch/ptracker/plasmid_graph/data/contigs_darwin_sub.fna"                           
# fnafile = sys.argv[1]
header2dna = {header: dna for dna, header in readfasta(fnafile)}

# Filter for only containing plasmids
# contigset = set(df.contigname)
# header2dna_filtered = {header: dna for header, dna in header2dna.items() if header in contigset}
#


0        >S3CS18097D-03-11_S0_L001CNODE_15981_length_31...
1        >S4CS18097D-03-12_S0_L001CNODE_6076_length_518...
2        >S4CS18097D-03-12_S0_L001CNODE_19427_length_25...
3        >S4CS18097D-03-12_S0_L001CNODE_19870_length_25...
4        >S2CS18097D-03-07_S0_L001CNODE_20253_length_23...
                               ...                        
12345    >S2CS18097D-03-07_S0_L001CNODE_15144_length_27...
12346    >S3CS18097D-03-11_S0_L001CNODE_18497_length_28...
12347    >S3CS18097D-03-11_S0_L001CNODE_18826_length_28...
12348    >S3CS18097D-03-11_S0_L001CNODE_28034_length_22...
12349    >S4CS18097D-03-12_S0_L001CNODE_22945_length_23...
Name: contigname, Length: 12350, dtype: object

In [53]:
!pip install click
import pandas as pd                                                              
import sys
import os

def readfasta(filename):
    '''Reading in several fasta files
       yield each pair of header and dna seperate
    '''
    # Try to open the file, error if file does not exsist
    try: file = open(filename, "r")
    except FileNotFoundError as errormessage:
        sys.exit(f"The file '{filename}' could not be found, error: {errormessage}")

    # Extract the dna, and the headers.
    oldheader = "FIRST HEADER"
    for line in file:
        line = line.strip()
        #if line is header yield dna and header except FIRST HEADER
        if line.startswith(">"):    
            newheader = line
            if oldheader != "FIRST HEADER":
                yield dna, oldheader
            dna = ""
            oldheader = newheader
        else:
            dna += line
    # Yield the last header and dna
    yield dna, oldheader
    file.close()

# clusterfile = sys.argv[0]
clusterfile_plasmid = "/home/bxc755/rasmussen/scratch/ptracker/plasmid_graph/data/vae_clusters_within_radius_with_looners_complete_unsplit_candidate_plasmids.tsv" 
clusterfile_chromosome = "/home/bxc755/rasmussen/scratch/ptracker/plasmid_graph/data/vae_clusters_within_radius_with_looners_complete_unsplit_candidate_plasmids.tsv" 
fasta_all_contigs = "/home/bxc755/rasmussen/scratch/ptracker/plasmid_graph/data/contigs_darwin_sub.fna"                           

df_chromosome = pd.read_table(clusterfile_chromosome, sep="\t")                                             
df_plasmid = pd.read_table(clusterfile_plasmid, sep="\t")                                             

df_plasmid = df_plasmid.assign(contigname = ">" + df_plasmid.contigname)
df_chromosome = df_chromosome.assign(contigname = ">" + df_chromosome.contigname)

# Get the sum of the lengths of each contig in each bin
df_chromosome = df_chromosome.assign(contig_length = df_chromosome.contigname.str.extract(r"length_(\d+)").astype("int"))
df_chromosome["cluster_length"] = df_chromosome.groupby("clustername")["contig_length"].transform("sum")

# Create 3 sets to filter fasta files later, containing different contigs
t = 200_000
chromosomes_above_t = set(df_chromosome.contigname[df_chromosome.cluster_length >= t])
chromosomes_below_t = set(df_chromosome.contigname[df_chromosome.cluster_length < t])
plasmids = set(df_plasmid.contigname)

plasmid_dir = "candidate_plasmids"
os.mkdir("candidate_plasmids")
chromosome_dir = "candidate_chromosomes"
os.mkdir("candidate_chromosomes")
# Write to the split fastafiles
with open(f'{plasmid_dir}/plasmids.fna', 'w') as plasmid_file, \
     open(f'{chromosome_dir}/chromosomes_candidate.fna', 'w') as chromosome_above_t, \
     open('chromosome_contigs_filtered_away.fna', 'w') as chromosome_below_t:
    for dna, header in readfasta(fasta_all_contigs ):
        if header in plasmids:
                print(header, file=plasmid_file)
                print(dna, file=plasmid_file)
        elif header in chromosomes_above_t:
                print(header, file=chromosomes_above_t)
                print(dna, file=chromosomes_above_t)
        elif header in chromosomes_below_t:
                print(header, file=chromosomes_below_t)
                print(dna, file=chromosomes_below_t)

FileExistsError: [Errno 17] File exists: 'candidate_plasmids'

4643